In [17]:
from lxml import html
import requests
from bs4 import BeautifulSoup
import re
import json

In [2]:
def get_movie_ids_from_list_page_soup(list_page_soup):
    """
    get movie_id from list_page_soup
    """
    movies = list_page_soup.find_all('div', class_="p-movie-cassette js-movie-cassette")
    
    if not movies:
        return False
    movie_ids = []
    
    for i in range(len(movies)):
        data_clip = movies[i].get('data-clip')
        movie_id = int(re.match(r'^.*"movie_id":(\d{5}).*', data_clip).group(1))
        movie_ids.append(movie_id)
        
    return movie_ids

In [3]:
def get_all_movie_ids(base_url):
    """
    get all movie ids from base url 
    (ex. https://filmarks.com/list/year/2010s/2018)
    """
    all_movie_ids = []
    page_number = 1
    while True :
        # request GET
        if page_number == 1:
            page = requests.get(base_url)
        else:
            payload = {'page': '{}'.format(page_number)}
            page = requests.get(base_url, params=payload)
        list_page_soup = BeautifulSoup(page.content, 'html.parser')
        
        movie_ids = get_movie_ids_from_list_page_soup(list_page_soup)
        if not movie_ids:
            break
        else:
            all_movie_ids.extend(movie_ids)
            page_number += 1
    return all_movie_ids

In [4]:
def get_reviews_from_individual_page_soup(individual_page_soup):
    """
    parse indivisual_page_soup and return reviews's list
    """
    reviews_obj = individual_page_soup.find_all('div', class_="p-mark__review")
    if not reviews_obj:
        return False
    else:
        return [reviews_obj[i].text for i in range(len(reviews_obj))]

In [5]:
def get_reviews_from_movie_id(movie_id):
    """
    get all reviews tied to a given movie_id
    and return a list of them
    """
    all_reviews = []
    base_url = 'https://filmarks.com/movies/' + str(movie_id)
    page_number = 1
    while True :
        # request GET
        if page_number == 1:
            page = requests.get(base_url)
        else:
            payload = {'page': '{}'.format(page_number)}
            page = requests.get(base_url, params=payload)
        individual_page_soup = BeautifulSoup(page.content, 'html.parser')
        
        reviews = get_reviews_from_individual_page_soup(individual_page_soup)
        if not reviews:
            break
        else:
            all_reviews.extend(reviews)
            page_number += 1
    return all_reviews

In [6]:
def get_title_from_movie_id(movie_id):
    base_url = 'https://filmarks.com/movies/' + str(movie_id)
    page = requests.get(base_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    title = soup.find('h2', class_="p-content-detail__title").span.text
    return title

In [19]:
def write_json(title, filmarks_reviews):
    data = {}
    data["title"] = title
    reviews = {}
    reviews["filmarks"] = filmarks_reviews
    data["reviews"] = reviews
    with open('jsons/{}.json'.format(title), "w", encoding="utf-8") as f:
        json.dump(data, f, sort_keys=True, indent=4, separators=(',', ': '))

In [ ]:
import csv
#print(all_movie_ids)
with open('movie_ids.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(all_movie_ids)

In [ ]:
for i in len(all_movie_ids):
    movie_id = all_movie_ids[i]
    title = get_title_from_movie_id(movie_id)
    reviews = get_reviews_from_movie_id(movie_id)
    

## TEST

In [ ]:
get_title_from_movie_id(79340)

In [20]:
write_json("おはよ", [1, 2, {'3': 3}, {'4': 4}])